In [ ]:
import math

import matplotlib.patches as patches
import matplotlib.pyplot as plt
from matplotlib.animation import FFMpegWriter, FuncAnimation

fig, ax = plt.subplots(figsize=(10, 10))


class Branch:
    def __init__(
        self,
        start: tuple[float, float],
        end: tuple[float, float],
        start_time: float,
        circle: float | None = None,
    ):
        self.start = start
        self.end = end
        self.start_time = start_time
        self.growth: float = 0
        self.circle = circle
        self.circle_growth: float = 0

        # Calculate length of the branch
        dx = end[0] - start[0]
        dy = end[1] - start[1]
        self.length = math.sqrt(dx * dx + dy * dy)

        if self.circle:
            self.circle = self.circle * RADIUS_SCALE

    def get_current_end(self) -> tuple[float, float]:
        x = self.start[0] + (self.end[0] - self.start[0]) * self.growth
        y = self.start[1] + (self.end[1] - self.start[1]) * self.growth
        return (x, y)


SPEED: float = 0.5  # Units per frame
LINEWIDTH: int = 14
RADIUS_SCALE: float = 0.5
CIRCILE_RATIO: float = 1.8
CIRCLE_ANIMATION_FRAMES = 20  # How many frames the circle animation takes


branches = [
    # centre
    Branch(start=(0, 0), end=(0, 50), start_time=0, circle=9),
    Branch(start=(0, 50), end=(0, 88), start_time=50, circle=10),
    Branch(start=(0, 50), end=(26, 79), start_time=50, circle=7.5),
    # right
    Branch(start=(0, 0), end=(41, 25), start_time=12, circle=11),
    Branch(start=(41, 25), end=(41, 60), start_time=48, circle=9),
    Branch(start=(41, 25), end=(21, 42), start_time=48, circle=6.5),
    # left
    Branch(start=(0, 0), end=(-41, 25), start_time=0, circle=14),
    Branch(start=(-41, 25), end=(-41, 60), start_time=48, circle=9.5),
    Branch(start=(-41, 25), end=(-20.5, 43), start_time=48),
    Branch(start=(-20.5, 43), end=(-20.5, 74), start_time=70, circle=7),
]


def circle_elastic_scale(growth: float) -> float:
    """Returns a scale factor that overshoots and settles back"""
    if growth < 0.5:  # Growing phase
        return 2 * growth * 1.3  # Overshoot by 30%
    else:  # Settling phase
        return 1 + 0.3 * (1 - (growth - 0.5) * 2)  # Settle back to 1


def draw_circles(
    ax, current_end: tuple[float, float], radius: float, circle_growth: float
) -> None:
    scale = circle_elastic_scale(circle_growth)

    outer_circle = patches.Circle(
        current_end,
        radius=radius * CIRCILE_RATIO * scale,
        alpha=0.6 * min(1, circle_growth * 2),
        color="white",
        linewidth=0,
    )
    ax.add_patch(outer_circle)
    inner_circle = patches.Circle(
        current_end,
        radius=radius * scale,
        alpha=min(1, circle_growth * 2),
        color="white",
    )
    ax.add_patch(inner_circle)


def update(frame: int) -> None:
    ax.clear()
    ax.set_axis_off()
    ax.set_xlim(-60, 60)
    ax.set_ylim(-15, 105)
    plt.tight_layout()

    current_distance = frame * SPEED

    for branch in branches:
        if current_distance >= branch.start_time:
            # Calculate growth based on distance traveled since start
            distance_along_branch = current_distance - branch.start_time
            branch.growth = min(1.0, distance_along_branch / branch.length)
            current_end = branch.get_current_end()

            # Draw lines
            line = plt.Line2D(
                [branch.start[0], current_end[0]],
                [branch.start[1], current_end[1]],
                linewidth=LINEWIDTH,
                color="white",
                solid_capstyle="round",
            )
            ax.add_line(line)

            # Draw circles with animation
            if branch.growth == 1.0 and branch.circle:
                excess_distance = distance_along_branch - branch.length
                branch.circle_growth = min(
                    1.0, excess_distance / (SPEED * CIRCLE_ANIMATION_FRAMES)
                )
                draw_circles(ax, current_end, branch.circle, branch.circle_growth)


def calculate_total_frames() -> int:
    max_distance = 0
    for branch in branches:
        path_distance = (
            branch.start_time + branch.length + (SPEED * CIRCLE_ANIMATION_FRAMES)
        )
        max_distance = max(max_distance, path_distance)
    return int(max_distance / SPEED) + 1


total_frames = calculate_total_frames()
anim = FuncAnimation(fig, update, frames=total_frames)
writer = FFMpegWriter(fps=100, metadata={"artist": "me"}, bitrate=2000)
anim.save(
    "logo_annimation.mp4",
    writer=writer,
    dpi=300,
    savefig_kwargs={"facecolor": "#60d4d4"},
)
plt.close()